In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import mediapipe as mp 
import cv2 as cv
import os 



In [3]:
## let's load the extracted landmarks 

file_path_Dataset='Top_Classes_Landmarks/Top_Classes_Landmarks'
file_path_Preprocessed='Top_Classes_Landmarks_Preprocessed_No_SlidingWindow_OR_Mask/Top_Classes_Landmarks_Preprocessed_No_SlidingWindow_OR_Mask'


Dataset= []
Dataset_glosses = []


for file in os.listdir(file_path_Dataset):
    if not file.endswith(".npy"):
        continue

    data = np.load(os.path.join(file_path_Dataset, file))
    label = file.split(' ')[0].lower() 

    Dataset.append(data)
    Dataset_glosses.append(label)

Dataset_preprocessed= []
Dataset_preprocessed_glosses = []



for file in os.listdir(file_path_Preprocessed):
    if not file.endswith(".npy"):
        continue

    data = np.load(os.path.join(file_path_Preprocessed, file))
    label = file.split('_')[0].lower()

    Dataset_preprocessed.append(data)
    Dataset_preprocessed_glosses.append(label)

print(len(Dataset_preprocessed), len(Dataset_preprocessed_glosses))





5076 5076


In [3]:
print(len(Dataset), len(Dataset_glosses))

5568 5568


In [4]:
Dataset_preprocessed[600].shape

(157, 438)

In [5]:
Dataset_glosses[7]

'about'

#### Preparing the dataset For feeding to the model 

In [4]:
from sklearn.model_selection import train_test_split

X_train_preprocessed, X_temp, y_train_preprocessed, y_temp = train_test_split(
    Dataset_preprocessed,
    Dataset_preprocessed_glosses,
    test_size=0.10,   
    random_state=42
)

X_val_preprocessed, X_test_preprocessed, y_val_preprocessed, y_test_preprocessed = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,  
    random_state=42
)

In [5]:


X_train_preprocessed = np.array(X_train_preprocessed)
X_val_preprocessed = np.array(X_val_preprocessed)
X_test_preprocessed = np.array(X_test_preprocessed)
y_train_preprocessed = np.array(y_train_preprocessed)
y_val_preprocessed = np.array(y_val_preprocessed)
y_test_preprocessed = np.array(y_test_preprocessed)

In [6]:
X_train_preprocessed.shape

(4568, 157, 438)

In [9]:
len(np.unique(Dataset_glosses))

292

In [10]:
len(np.unique(Dataset_preprocessed_glosses))

132

####  sequence-to-sequence (encoder–decoder) model with attention

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, GRU, Dense
from tensorflow.keras.models import Model

# =========================
# PARAMETERS
# =========================
T = X_train_preprocessed.shape[1]   # encoder timesteps
D = X_train_preprocessed.shape[2]   # feature dimension

ENC_UNITS = 1000
VOCAB_SIZE = 132     # number of target words (NOT glosses)

# =========================
# ENCODER ONLY MODEL
# =========================
encoder_inputs = Input(shape=(T, D), name="encoder_inputs")

# GRU encoder (return the last state)
encoder_output = GRU(
    ENC_UNITS,
    dropout=0.2
)(encoder_inputs)  # shape: (batch_size, ENC_UNITS)

# Output layer for classification
output = Dense(
    VOCAB_SIZE,
    activation='softmax'
)(encoder_output)

# Build model
model = Model(inputs=encoder_inputs, outputs=output)





In [8]:
from sklearn.preprocessing import LabelEncoder


# Create label encoder
le = LabelEncoder()

# Fit on training labels and transform
y_train_encoded = le.fit_transform(y_train_preprocessed)
y_val_encoded   = le.transform(y_val_preprocessed)
y_test_encoded  = le.transform(y_test_preprocessed)


In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_inputs (InputLayer)     │ (None, 157, 438)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 1000)           │     4,320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 132)            │       132,132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,452,132 (16.98 MB)

 Trainable params: 4,452,132 (16.98 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
history = model.fit(
    X_train_preprocessed,      # encoder input
    y_train_encoded,           # target word labels
    validation_data=(X_val_preprocessed, y_val_encoded),
    batch_size=32,
    epochs=100
)




Epoch 1/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 123s 859ms/step - accuracy: 0.0252 - loss: 4.8582 - val_accuracy: 0.0236 - val_loss: 4.7669
Epoch 2/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 128s 898ms/step - accuracy: 0.0258 - loss: 4.8133 - val_accuracy: 0.0433 - val_loss: 4.7774
Epoch 3/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 128s 896ms/step - accuracy: 0.0313 - loss: 4.7772 - val_accuracy: 0.0394 - val_loss: 4.7880
Epoch 4/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 124s 866ms/step - accuracy: 0.0302 - loss: 4.7568 - val_accuracy: 0.0354 - val_loss: 4.7630
Epoch 5/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 121s 846ms/step - accuracy: 0.0333 - loss: 4.7171 - val_accuracy: 0.0472 - val_loss: 4.7242
Epoch 6/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 125s 875ms/step - accuracy: 0.0412 - loss: 4.5225 - val_accuracy: 0.0276 - val_loss: 4.3882
Epoch 7/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 124s 868ms/step - accuracy: 0.0532 - loss: 4.2762 - val_accuracy: 0.0433 - val_loss: 4.2980
Epoch 8/100
143/143 ━━━━━━━━━━━━━━━━━━━━ 124s 868ms/step - accuracy: 

In [12]:
test_loss, test_acc = model.evaluate(X_test_preprocessed, y_test_encoded)
print("Test accuracy:", test_acc)


8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 206ms/step - accuracy: 0.5906 - loss: 1.8193
Test accuracy: 0.5905511975288391


### Before preprocessing 

In [92]:
# Count how many samples per word
from collections import Counter


label_counts = Counter(Dataset_glosses)

MIN_SAMPLES = 2  # must be >=2 for stratification

valid_labels = {
    label for label, count in label_counts.items()
    if count >= MIN_SAMPLES
}

print("Original samples:", len(Dataset))
print("Original vocab size:", len(label_counts))

# Filter dataset
Dataset_filtered = []
Glosses_filtered = []

for x, y in zip(Dataset, Dataset_glosses):
    if y in valid_labels:
        Dataset_filtered.append(x)
        Glosses_filtered.append(y)

print("Filtered samples:", len(Dataset_filtered))
print("Filtered vocab size:", len(set(Glosses_filtered)))


Original samples: 5568
Original vocab size: 292
Filtered samples: 5422
Filtered vocab size: 146


In [93]:
X_train_raw, X_temp, y_train_raw, y_temp = train_test_split(
    Dataset_filtered,
    Glosses_filtered,
    test_size=0.10,
    random_state=42,
    stratify=Glosses_filtered
)

X_val_raw, X_test_raw, y_val_raw, y_test_raw = train_test_split(
    X_temp,
    y_temp,
    test_size=0.50,
    random_state=42,
    stratify=y_temp
)


In [94]:
MAX_LEN = 64

def pad_or_truncate(sequence, max_len=64):
    T, D = sequence.shape
    if T > max_len:
        return sequence[:max_len]
    elif T < max_len:
        padding = np.zeros((max_len - T, D))
        return np.vstack([sequence, padding])
    return sequence


In [ ]:
X_train_raw = np.array([pad_or_truncate(seq, MAX_LEN) for seq in X_train_raw])
X_val_raw   = np.array([pad_or_truncate(seq, MAX_LEN) for seq in X_val_raw])
X_test_raw  = np.array([pad_or_truncate(seq, MAX_LEN) for seq in X_test_raw])

print("X_train shape:", X_train_raw.shape)  # (N, 64, D)
print("X_val shape:", X_val_raw.shape)  # (N, 64, D)
print("X_test shape:", X_test_raw.shape)  # (N, 64, D)

X_train shape: (4879, 64, 438)
X_val shape: (271, 64, 438)
X_test shape: (272, 64, 438)


In [98]:
le = LabelEncoder()

y_train = le.fit_transform(y_train_raw)
y_val   = le.transform(y_val_raw)
y_test  = le.transform(y_test_raw)

In [96]:


# =========================
# PARAMETERS
# =========================
T = X_train_raw.shape[1]   # encoder timesteps
D = X_train_raw.shape[2]   # feature dimension

ENC_UNITS = 1000
VOCAB_SIZE = 146    # number of target words (NOT glosses)

# =========================
# ENCODER ONLY MODEL
# =========================
encoder_inputs = Input(shape=(T, D), name="encoder_inputs")

# GRU encoder (return the last state)
encoder_output = GRU(
    ENC_UNITS,
    dropout=0.2
)(encoder_inputs)  # shape: (batch_size, ENC_UNITS)

# Output layer for classification
output = Dense(
    VOCAB_SIZE,
    activation='softmax'
)(encoder_output)

# Build model
model_raw = Model(inputs=encoder_inputs, outputs=output)

In [97]:
model_raw.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "functional_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_inputs (InputLayer)     │ (None, 157, 438)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_16 (GRU)                    │ (None, 1000)           │     4,320,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 132)            │       132,132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,452,132 (16.98 MB)

 Trainable params: 4,452,132 (16.98 MB)

 Non-trainable params: 0 (0.00 B)

In [99]:
history = model_raw.fit(
    X_train_raw,      # encoder input
    y_train,           # target word labels
    validation_data=(X_val_raw, y_val),
    batch_size=32,
    epochs=50
)


Epoch 1/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 61s 385ms/step - accuracy: 0.0172 - loss: 4.9813 - val_accuracy: 0.0258 - val_loss: 4.9330
Epoch 2/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 57s 375ms/step - accuracy: 0.0258 - loss: 4.9226 - val_accuracy: 0.0332 - val_loss: 4.9306
Epoch 3/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 55s 361ms/step - accuracy: 0.0324 - loss: 4.8085 - val_accuracy: 0.0221 - val_loss: 4.7359
Epoch 4/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 56s 366ms/step - accuracy: 0.0437 - loss: 4.5540 - val_accuracy: 0.0443 - val_loss: 4.5475
Epoch 5/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 57s 374ms/step - accuracy: 0.0432 - loss: 4.4424 - val_accuracy: 0.0517 - val_loss: 4.3821
Epoch 6/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 56s 369ms/step - accuracy: 0.0547 - loss: 4.3173 - val_accuracy: 0.0554 - val_loss: 4.2814
Epoch 7/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 56s 364ms/step - accuracy: 0.0549 - loss: 4.2020 - val_accuracy: 0.0590 - val_loss: 4.1903
Epoch 8/50
153/153 ━━━━━━━━━━━━━━━━━━━━ 58s 380ms/step - accuracy: 0.0642 - loss: 4

In [100]:
test_loss, test_acc = model_raw.evaluate(X_test_raw, y_test)
print("Test accuracy:", test_acc)


9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 81ms/step - accuracy: 0.4706 - loss: 2.1804
Test accuracy: 0.47058823704719543
